# Minibatch Stochastic Gradient Descent
:label:`sec_minibatch_sgd`

So far we encountered two extremes in the approach to gradient based learning: :numref:`sec_gd` uses the full dataset to compute gradients and to update parameters, one pass at a time. Conversely :numref:`sec_sgd` processes one observation at a time to make progress. Each of them has its own drawbacks. Gradient Descent is not particularly *data efficient* whenever data is very similar. Stochastic Gradient Descent is not particularly *computationally efficient* since CPUs and GPUs cannot exploit the full power of vectorization. This suggests that there might be a happy medium, and in fact, that's what we have been using so far in the examples we discussed. 

## Vectorization and Caches

At the heart of the decision to use minibatches is computational efficiency. This is most easily understood when considering parallelization to multiple GPUs and multiple servers. In this case we need to send at least one image to each GPU. With 8 GPUs per server and 16 servers we already arrive at a minibatch size of 128.

Things are a bit more subtle when it comes to single GPUs or even CPUs. These devices have multiple types of memory, often multiple type of compute units and different bandwidth constraints between them. For instance, a CPU has a small number of registers and then L1, L2 and in some cases even L3 cache (which is shared between the different processor cores). These caches are of increasing size and latency (and at the same time they are of decreasing bandwidth). Suffice it to say, the processor is capable of performing many more operations than what the main memory interface is able to provide. 

* A 2GHz CPU with 16 cores and AVX-512 vectorization can process up to $2 \cdot 10^9 \cdot 16 \cdot 32 = 10^{12}$ bytes per second. The capability of GPUs easily exceeds this number by a factor of 100. On the other hand, a midrange server processor might not have much more than 100 GB/s bandwidth, i.e., less than one tenth of what would be required to keep the processor fed. To make matters worse, not all memory access is created equal: first, memory interfaces are typically 64 bit wide or wider (e.g., on GPUs up to 384 bit), hence reading a single byte incurs the cost of a much wider access. 
* There is significant overhead for the first access whereas sequential access is relatively cheap (this is often called a burst read). There are many more things to keep in mind, such as caching when we have multiple sockets, chiplets and other structures. A detailed discussion of this is beyond the scope of this section. See e.g., this [Wikipedia article](https://en.wikipedia.org/wiki/Cache_hierarchy) for a more in-depth discussion.

The way to alleviate these constraints is to use a hierarchy of CPU caches which are actually fast enough to supply the processor with data. This is *the* driving force behind batching in deep learning. To keep matters simple, consider matrix-matrix multiplication, say $\mathbf{A} = \mathbf{B}\mathbf{C}$. We have a number of options for calculating $\mathbf{A}$. For instance we could try the following:

1. We could compute $\mathbf{A}_{ij} = \mathbf{B}_{i,:} \mathbf{C}_{:,j}^\top$, i.e., we could compute it element-wise by means of dot products.
1. We could compute $\mathbf{A}_{:,j} = \mathbf{B} \mathbf{C}_{:,j}^\top$, i.e., we could compute it one column at a time. Likewise we could compute $\mathbf{A}$ one row $\mathbf{A}_{i,:}$ at a time. 
1. We could simply compute $\mathbf{A} = \mathbf{B} \mathbf{C}$. 
1. We could break $\mathbf{B}$ and $\mathbf{C}$ into smaller block matrices and compute $\mathbf{A}$ one block at a time. 

If we follow the first option, we will need to copy one row and one column vector into the CPU each time we want to compute an element $\mathbf{A}_{ij}$. Even worse, due to the fact that matrix elements are aligned sequentially we are thus required to access many disjoint locations for one of the two vectors as we read them from memory. The second option is much more favorable. In it, we are able to keep the column vector $\mathbf{C}_{:,j}$ in the CPU cache while we keep on traversing through $B$. This halves the memory bandwidth requirement with correspondingly faster access. Of course, option 3 is most desirable. Unfortunately, most matrices might not entirely fit into cache (this is what we are discussing after all). However, option 4 offers a practically useful alternative: we can move blocks of the matrix into cache and multiply them locally. Optimized libraries take care of this for us. Let us have a look at how efficient these operations are in practice.

In [1]:
%use @file[../djl-pytoch-0.18.0.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
//import jp.live.ugai.d2j.attention.Chap10Utils
import org.jetbrains.letsPlot.intern.Plot
import jp.live.ugai.d2j.util.StopWatch
import jp.live.ugai.d2j.util.Training
import jp.live.ugai.d2j.util.Accumulator
// %load ../utils/djl-imports
// %load ../utils/plot-utils
// %load ../utils/Functions.java
// %load ../utils/StopWatch.java
// %load ../utils/Training.java
// %load ../utils/Accumulator.java

In [2]:
import ai.djl.basicdataset.tabular.*
import ai.djl.basicdataset.cv.classification.*
// import org.apache.commons.lang3.ArrayUtils;

In [3]:
var manager = NDManager.newBaseManager();
val stopWatch = StopWatch();
var A = manager.zeros(Shape(256, 256));
val B = manager.randomNormal(Shape(256, 256));
val C = manager.randomNormal(Shape(256, 256));

Element-wise assignment simply iterates over all rows and columns of $\mathbf{B}$ and $\mathbf{C}$ respectively to assign the value to $\mathbf{A}$.

In [4]:
// Compute A = B C one element at a time
stopWatch.start();
for (i in 0 until 256) {
    for (j in 0 until 256) {
        A.set(NDIndex(i.toLong(), j.toLong()), 
              B.get(NDIndex("$i, :")).dot(C.get(NDIndex(":, $j"))))
    }
}
stopWatch.stop()

2.1147689

A faster strategy is to perform column-wise assignment.

In [5]:
// Compute A = B C one column at a time
stopWatch.start();
for (j in 0 until 256) {
    A.set(NDIndex(":, $j"), B.matMul(C.get(NDIndex(":, $j"))))
}
stopWatch.stop();

0.0248393

Last, the most effective manner is to perform the entire operation in one block. Let us see what the respective speed of the operations is.

In [6]:
// Compute A = B C in one go
stopWatch.start();
A = B.dot(C);
stopWatch.stop();

// Multiply and add count as separate operations (fused in practice)
val gigaflops = mutableListOf<Float>()
for (i in 0 until stopWatch.times.size) {
    gigaflops.add(2 / stopWatch.times.get(i).toFloat())
}
"Performance in Gigaflops: element %.3f, column %.3f, full %.3f".format(gigaflops[0], gigaflops[1], gigaflops[2])

Performance in Gigaflops: element 0.946, column 80.518, full 1293.996

## Minibatches 

:label:`sec_minibatches`

In the past we took it for granted that we would read *minibatches* of data rather than single observations to update parameters. We now give a brief justification for it. Processing single observations requires us to perform many single matrix-vector (or even vector-vector) multiplications, which is quite expensive and which incurs a significant overhead on behalf of the underlying deep learning framework. This applies both to evaluating a network when applied to data (often referred to as inference) and when computing gradients to update parameters. That is, this applies whenever we perform $\mathbf{w} \leftarrow \mathbf{w} - \eta_t \mathbf{g}_t$ where 

$$\mathbf{g}_t = \partial_{\mathbf{w}} f(\mathbf{x}_{t}, \mathbf{w})$$

We can increase the *computational* efficiency of this operation by applying it to a minibatch of observations at a time. That is, we replace the gradient $\mathbf{g}_t$ over a single observation by one over a small batch

$$\mathbf{g}_t = \partial_{\mathbf{w}} \frac{1}{|\mathcal{B}_t|} \sum_{i \in \mathcal{B}_t} f(\mathbf{x}_{i}, \mathbf{w})$$

Let us see what this does to the statistical properties of $\mathbf{g}_t$: since both $\mathbf{x}_t$ and also all elements of the minibatch $\mathcal{B}_t$ are drawn uniformly at random from the training set, the expectation of the gradient remains unchanged. The variance, on the other hand, is reduced significantly. Since the minibatch gradient is composed of $b := |\mathcal{B}_t|$ independent gradients which are being averaged, its standard deviation is reduced by a factor of $b^{-\frac{1}{2}}$. This, by itself, is a good thing, since it means that the updates are more reliably aligned with the full gradient. 

Naively this would indicate that choosing a large minibatch $\mathcal{B}_t$ would be universally desirable. Alas, after some point, the additional reduction in standard deviation is minimal when compared to the linear increase in computational cost. In practice we pick a minibatch that is large enough to offer good computational efficiency while still fitting into the memory of a GPU. To illustrate the savings let us have a look at some code. In it we perform the same matrix-matrix multiplication, but this time broken up into "minibatches" of 64 columns at a time.

In [7]:
stopWatch.start();
for (j in 0 until 256 step 64 ) {
    A.set(NDIndex(":, $j:${j+64}"), 
        B.dot(C.get(NDIndex(":, $j:${j+64}"))))
}
stopWatch.stop();

"Performance in Gigaflops: block %.3f\n".format(2 / stopWatch.times.get(3))

Performance in Gigaflops: block 107.494


As we can see, the computation on the minibatch is essentially as efficient as on the full matrix. A word of caution is in order. In :numref:`sec_batch_norm` we used a type of regularization that was heavily dependent on the amount of variance in a minibatch. As we increase the latter, the variance decreases and with it the benefit of the noise-injection due to batch normalization. See e.g., :cite:`Ioffe.2017` for details on how to rescale and compute the appropriate terms. 

## Reading the Dataset

Let us have a look at how minibatches are efficiently generated from data. In the following we use a dataset developed by NASA to test the wing [noise from different aircraft](https://archive.ics.uci.edu/ml/datasets/Airfoil+Self-Noise) to compare these optimization algorithms. For convenience we only use the first $1,500$ examples. The data is whitened for preprocessing, i.e., we remove the mean and rescale the variance to $1$ per coordinate.

In [8]:
manager = NDManager.newBaseManager();

fun getDataCh11(batchSize: Int, n: Int) : AirfoilRandomAccess {
    // Load data
    val airfoil = AirfoilRandomAccess.builder()
            .optUsage(Dataset.Usage.TRAIN)
            .setSampling(batchSize, true)
            .optNormalize(true)
            .optLimit(n.toLong())
            .build()
    return airfoil
}


## Implementation from Scratch

Recall the minibatch SGD implementation from :numref:`sec_linear_scratch`. In the following we provide a slightly more general implementation. For convenience it has the same call signature as the other optimization algorithms introduced later in this chapter. Specifically, we add the status
input `states` and place the hyperparameter in dictionary `hyperparams`. In
addition, we will average the loss of each minibatch example in the training
function, so the gradient in the optimization algorithm does not need to be
divided by the batch size.

In [9]:
object Optimization {
    fun sgd(params: NDList, states: NDList, hyperparams: Map<String, Float>) {
        val lrt = Tracker.fixed(hyperparams.get("lr")!!)
        val opt = Optimizer.sgd().setLearningRateTracker(lrt).build()
        for (param in params) {
            // Update param in place.
            // param = param - param.gradient * lr / batchSize
            // val ind = params.indexOf(param)
            // params.rep
            // params.set(ind, param.sub(param.getGradient().mul(lr).div(batchSize)))
            opt.update(param.toString(), param, param.gradient)
//            param.subi(param.getGradient().mul(lr).div(batchSize));
        }

//        for (i in 0 until params.size) {
//            val param = params.get(i);
            // Update param
            // param = param - param.gradient * lr
//            param.subi(param.getGradient().mul(hyperparams.get("lr")));
//        }
    }
}

Next, we implement a generic training function to facilitate the use of the other optimization algorithms introduced later in this chapter. It initializes a linear regression model and can be used to train the model with minibatch SGD and other algorithms introduced subsequently.

In [10]:
fun evaluateLoss( dataIterator: Iterable<Batch>, w: NDArray, b: NDArray): Float {
    val metric = Accumulator(2);  // sumLoss, numExamples

    for (batch in dataIterator) {
        val X = batch.getData().head();
        val y = batch.getLabels().head();
        val yHat = Training.linreg(X, w, b);
        val lossSum = Training.squaredLoss(yHat, y).sum().getFloat();

        metric.add(floatArrayOf(lossSum, y.size().toFloat()))
        batch.close()
    }
    return metric.get(0) / metric.get(1);
}


In [11]:
class LossTime(val epoch: List<Double>, val loss: List<Float>, val time: List<Double>)

In [12]:
fun plotLossEpoch(loss: List<Float>, epoch: List<Double>) : Plot {
    val dd = mapOf( "x" to epoch, "y" to loss)
var plot = letsPlot()
plot += geomLine(data=dd) { x = "x" ; y = "y" }
return plot + ggsize(500, 400)
}

In [13]:
fun trainCh11(trainer: (NDList, NDList, Map<String, Float>)-> Unit,
              states: NDList,
              hyperparams: Map<String, Float> ,
              dataset: AirfoilRandomAccess ,
              featureDim: Int,
              numEpochs: Int) : LossTime {
    val manager = NDManager.newBaseManager();
    val w = manager.randomNormal(0f, 0.01f, Shape(featureDim.toLong(), 1), DataType.FLOAT32);
    val b = manager.zeros(Shape(1));

    w.setRequiresGradient(true);
    b.setRequiresGradient(true);

    val params = NDList(w, b);
    var n = 0;
    val stopWatch = StopWatch();
    stopWatch.start();

    var lastLoss = -1.0f;
    val loss = mutableListOf<Float>();
    val epoch = mutableListOf<Double>()
    
    for (i in 0 until numEpochs) {
        for (batch in dataset.getData(manager)) {
            val len = (dataset.size() / batch.getSize()).toInt()  // number of batches
            val X = batch.getData().head();
            val y = batch.getLabels().head();

            val gc = Engine.getInstance().newGradientCollector()
                val yHat = Training.linreg(X, params.get(0), params.get(1));
                val l = Training.squaredLoss(yHat, y).mean();
                gc.backward(l);

            trainer(params, states, hyperparams);
            n += X.getShape().get(0).toInt()

            if (n % 200 == 0) {
                stopWatch.stop();
                lastLoss = evaluateLoss(dataset.getData(manager), params.get(0), params.get(1));
                loss.add(lastLoss);
                val lastEpoch = 1.0 * n / X.getShape().get(0) / len
                epoch.add(lastEpoch);
                stopWatch.start()
            }

            batch.close();
        }
    }
    System.out.printf("loss: %.3f, %.3f sec/epoch\n", lastLoss, stopWatch.avg());
//    plotLossEpoch(lossArray, epochArray);
    return LossTime(epoch, loss, stopWatch.cumsum());
}

Let us see how optimization proceeds for batch gradient descent. This can be achieved by setting the minibatch size to 1500 (i.e., to the total number of examples). As a result the model parameters are updated only once per epoch. There is little progress. In fact, after 6 steps progress stalls.

In [24]:
fun  trainSgd(lr: Float, batchSize: Int,numEpochs: Int) : LossTime {
    val dataset = getDataCh11(batchSize, 1500);
    val featureDim = dataset.getColumnNames().size

    val hyperparams = mutableMapOf<String, Float>()
    hyperparams.put("lr", lr);

    return trainCh11(Optimization::sgd, NDList(), hyperparams, dataset, featureDim, numEpochs);
}

val gdRes = trainSgd(1f, 1500, 10)
plotLossEpoch(gdRes.loss, gdRes.epoch)

loss: 0.246, 0.032 sec/epoch


When the batch size equals 1, we use SGD for optimization. For simplicity of implementation we picked a constant (albeit small) learning rate. In SGD, the model parameters are updated whenever an example is processed. In our case this amounts to 1500 updates per epoch. As we can see, the decline in the value of the objective function slows down after one epoch. Although both the procedures processed 1500 examples within one epoch, SGD consumes more time than gradient descent in our experiment. This is because SGD updated the parameters more frequently and since it is less efficient to process single observations one at a time.

In [25]:
val sgdRes = trainSgd(0.005f, 1, 2);
plotLossEpoch(sgdRes.loss, sgdRes.epoch)

loss: 0.244, 0.089 sec/epoch


Last, when the batch size equals 100, we use minibatch SGD for optimization. The time required per epoch is longer than the time needed for SGD and the time for batch gradient descent.

In [26]:
val mini1Res = trainSgd(0.4f, 100, 2);
plotLossEpoch(mini1Res.loss, mini1Res.epoch)

loss: 0.244, 0.005 sec/epoch


Reducing the batch size to 10, the time for each epoch increases because the workload for each batch is less efficient to execute.

In [27]:
val mini2Res = trainSgd(0.05f, 10, 2);
plotLossEpoch(mini2Res.loss, mini2Res.epoch)

loss: 0.244, 0.010 sec/epoch


Finally, we compare the time versus loss for the preview four experiments. As can be seen, despite SGD converges faster than GD in terms of number of examples processed, it uses more time to reach the same loss than GD because that computing gradient example by example is not efficient. Minibatch SGD is able to trade-off the convergence speed and computation efficiency. A minibatch size 10 is more efficient than SGD; a minibatch size 100 even outperforms GD in terms of runtime.

In [23]:
fun getTypeArray(lossTime: LossTime ,  name: String) : List<String> {
    return List<String>(lossTime.time.size) { name }
}

In [32]:
// Converts a float array to a log scale
fun convertLogScale(array: List<Double>) : List<Double>{
    return array.map { kotlin.math.log10(it)}
}

In [33]:
val time = convertLogScale(gdRes.time+ sgdRes.time + mini1Res.time+ mini2Res.time)
val loss = gdRes.loss+ sgdRes.loss + mini1Res.loss+ mini2Res.loss
val type = getTypeArray(gdRes, "gd") + getTypeArray(sgdRes, "sgd") +getTypeArray(mini1Res, "batch size = 100")+getTypeArray(mini1Res, "batch size = 10")
    val dd = mapOf( "x" to time, "y" to loss, "z" to type)
var plot = letsPlot()
plot += geomLine(data=dd) { x = "x" ; y = "y" ; color = "z"}
plot + ggsize(700, 400)


## Concise Implementation

In DJL, we can use the `Optimizer` package to access different optimization algorithms. This is used to implement a generic training function. We will use this throughout the current chapter.

In [38]:
import ai.djl.metric.Metrics
fun trainConciseCh11(sgd:Optimizer , dataset:AirfoilRandomAccess , 
                            numEpochs: Int) : kotlin.Pair<List<Double>, List<Float>> {
    // Initialization
    val manager = NDManager.newBaseManager();

    val net = SequentialBlock();
    val linear = Linear.builder().setUnits(1).build();
    net.add(linear);
    net.setInitializer(NormalInitializer(), Parameter.Type.WEIGHT);

    val model = Model.newInstance("concise implementation");
    model.setBlock(net);

    val loss = Loss.l2Loss();

    val config = DefaultTrainingConfig(loss)
        .optOptimizer(sgd)
        .addEvaluator(Accuracy()) // Model Accuracy
        .addTrainingListeners(*TrainingListener.Defaults.logging()) // Logging

    val trainer = model.newTrainer(config);

    var n = 0L;
    val stopWatch = StopWatch();
    stopWatch.start();

    trainer.initialize(Shape(10, 5));

    val metrics = Metrics();
    trainer.setMetrics(metrics);

    var lastLoss = -1.0f;
    
    val lossArray = mutableListOf<Float>()
    val epochArray = mutableListOf<Double>()
    for (epoch in 0 until numEpochs) {
    for (batch in trainer.iterateDataset(dataset)) {
        val len = dataset.size() / batch.getSize()  // number of batches

        val X = batch.getData().head();
        EasyTrain.trainBatch(trainer, batch);
        trainer.step();
        
        n += X.getShape().get(0);
        
        if (n % 200 == 0L) {
            stopWatch.stop();
            stopWatch.stop();
            lastLoss = evaluateLoss(dataset.getData(manager), linear.getParameters().get(0).getValue().getArray()
                            .reshape(Shape(dataset.getColumnNames().size.toLong(), 1L)),
                    linear.getParameters().get(1).getValue().getArray());
            lossArray.add(lastLoss);
            val lastEpoch = 1.0 * n / X.getShape().get(0) / len;
            epochArray.add(lastEpoch);
            stopWatch.start();
        }
        batch.close();
    }
    }
//    plotLossEpoch(arrayListToFloat(lossArray), arrayListToFloat(epochArray));

    System.out.printf("loss: %.3f, %.3f sec/epoch\n", lastLoss, stopWatch.avg());
    return kotlin.Pair(epochArray, lossArray)
}

Using DJL to repeat the last experiment shows identical behavior.

In [39]:
val airfoilDataset = getDataCh11(10, 1500);

val lrt = Tracker.fixed(0.05f);
val sgd = Optimizer.sgd().setLearningRateTracker(lrt).build();

val res = trainConciseCh11(sgd, airfoilDataset, 2)
plotLossEpoch(res.second, res.first)

Training:    100% |████████████████████████████████████████| Accuracy: 0.68, L2Loss: 0.27�███                                  | Accuracy: 0.69, L2Loss: 0.34racy: 0.69, L2Loss: 0.34              | Accuracy: 0.67, L2Loss: 0.32curacy: 0.67, L2Loss: 0.32��███                   | Accuracy: 0.67, L2Loss: 0.31��█████████████████               | Accuracy: 0.68, L2Loss: 0.30�███████████████████            | Accuracy: 0.68, L2Loss: 0.30     75% |███████████████████████████████         | Accuracy: 0.67, L2Loss: 0.30�████████████████████████     | Accuracy: 0.67, L2Loss: 0.28�███████████████████  | Accuracy: 0.67, L2Loss: 0.28Loss: 0.28
Training:    100% |████████████████████████████████████████| Accuracy: 0.67, L2Loss: 0.26�███                                  | Accuracy: 0.67, L2Loss: 0.27��██                              | Accuracy: 0.68, L2Loss: 0.27              | Accuracy: 0.68, L2Loss: 0.27curacy: 0.68, L2Loss: 0.27                | Accuracy: 0.68, L2Loss: 0.270.27ining:     68% |█████████

## Summary

* Vectorization makes code more efficient due to reduced overhead arising from the deep learning framework and due to better memory locality and caching on CPUs and GPUs. 
* There is a trade-off between statistical efficiency arising from SGD and computational efficiency arising from processing large batches of data at a time. 
* Minibatch stochastic gradient descent offers the best of both worlds: computational and statistical efficiency. 
* In minibatch SGD we process batches of data obtained by a random permutation of the training data (i.e., each observation is processed only once per epoch, albeit in random order). 
* It is advisable to decay the learning rates during training. 
* In general, minibatch SGD is faster than SGD and gradient descent for convergence to a smaller risk, when measured in terms of clock time.  

## Exercises

1. Modify the batch size and learning rate and observe the rate of decline for the value of the objective function and the time consumed in each epoch.
1. Read the DJL documentation and explore the different learning rate trackers in `ai.djl.training.optimizer.tracker` to see how they affect training. Try using a `FactorTracker` to reduce the learning rate to 1/10 of its previous value after each epoch.
1. Compare minibatch SGD with a variant that actually *samples with replacement* from the training set. What happens?
1. An evil genie replicates your dataset without telling you (i.e., each observation occurs twice and your dataset grows to twice its original size, but nobody told you). How does the behavior of SGD, minibatch SGD and that of gradient descent change?